In [ ]:
#include <vector>
#include <array>
#include <iostream>
#include <thread>
#include <chrono>
#include <random>
#include "xcpp/xdisplay.hpp"

struct game_of_life {
public:
    game_of_life(int width, int height)
        : shape{width, height}, 
          cells{std::vector<uint8_t>(width * height, 0), std::vector<uint8_t>(width * height, 0)},
          offsets{
            -width, width, -1, 1,
            // diagonals
            -width -1, -width +1, width -1, width +1
        }
    {
    }
    std::array<int,2> shape;
    std::array<int, 8> offsets;
    std::array<std::vector<uint8_t>, 2> cells;
    std::vector<uint8_t> & get_cells() {
        return cells[m_flip ? 1 : 0];
    }
    bool m_flip = false;
    void step() {
        auto & current_cells = cells[m_flip ? 0 : 1];
        auto & new_cells = cells[m_flip ? 1 : 0];   
        m_flip = !m_flip;
        for(auto i=0; i<current_cells.size(); ++i) {
            int live_neighbors = 0;
            for(auto offset : offsets) {
                int neighbor_index = i + offset;
                live_neighbors += neighbor_index >= 0 && neighbor_index < current_cells.size() ? 
                                    current_cells[neighbor_index] : 0;

            }
            new_cells[i] = current_cells[i] == 1 ? 
                           (live_neighbors == 2 || live_neighbors == 3 ? 1 : 0) : 
                           (live_neighbors == 3 ? 1 : 0);
        }   
    }
    bool is_alive(int x, int y) const {
        return cells[m_flip ? 0 : 1][y * shape[0] + x] == 1;
    }
    void set_value(int x, int y, uint8_t value) {
        cells[m_flip ? 0 : 1][y * shape[0] + x] = value;
    }
};

// overload << for game_of_life
std::ostream & operator<<(std::ostream & os, const game_of_life & gol) {
    for (int x = 0; x < gol.shape[0]+2; ++x) {
        os << "-";
    }
    os << '\n';
    for (int y = 0; y < gol.shape[1]; ++y) {
        std::cout<<"|";
        for (int x = 0; x < gol.shape[0]; ++x) {
            os << (gol.is_alive(x, y) ? '*' : ' ');
        }
        std::cout<<"|";
        os << '\n';
    }
    for (int x = 0; x < gol.shape[0]+2; ++x) {
        os << "-";
    }
    os << '\n';
    return os;
    
}
game_of_life gol(120,60);
// initialize with random values
std::random_device rd;
std::mt19937 gen(rd());
std::bernoulli_distribution dist(0.5);

for(int y=0; y<gol.shape[1]; ++y){
    for(int x=0; x<gol.shape[0]; ++x){
        gol.set_value(x,y, dist(gen));
    }
}

for (int i=0; i<30 ;++i){
    gol.step();
    std::cout << gol << std::endl;

    // wait a bit
    std::this_thread::sleep_for(std::chrono::milliseconds(100));
    xcpp::clear_output(true);  // prevents flickering

}